In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df=pd.read_csv(r"E:\Data Scientist\Amazon\Comment\comment.csv",encoding= 'unicode_escape')
df

,Translated_Review,Sentiment
0,I like eat delicious food. That's I'm cooking ...,Positive
1,This help eating healthy exercise regular basis,Positive
2,NaN,NaN
3,Works great especially going grocery store,Positive
4,Best idea us,Positive
...,...,...
64290,NaN,NaN
64291,NaN,NaN
64292,NaN,NaN
64293,NaN,NaN


In [7]:
df.isna().values.sum()

53731

In [3]:
df1=df.dropna()
df1

,Translated_Review,Sentiment
0,I like eat delicious food. That's I'm cooking ...,Positive
1,This help eating healthy exercise regular basis,Positive
3,Works great especially going grocery store,Positive
4,Best idea us,Positive
5,Best way,Positive
...,...,...
64222,Most ads older many agents ..not much owner po...,Positive
64223,"If photos posted portal load, fit purpose. I'm...",Positive
64226,"Dumb app, I wanted post property rent give opt...",Negative
64227,I property business got link SMS happy perform...,Positive


In [4]:
df1.isna().values.sum()

0

In [12]:
df1.groupby('Sentiment').describe()

Translated_Review                    
                      count unique     top freq
Sentiment                                      
Negative               8271   6240  I hate   14
Neutral                5158   4161  I like   56
Positive              23998  17593    Good  247

In [5]:
target=[0,1,2]
f=df1['Sentiment'].apply( lambda x: 0 if x=='Negative' else ( 1 if x=='Positive' else 2 )  ) 
f

0        1
1        1
3        1
4        1
5        1
        ..
64222    1
64223    1
64226    0
64227    1
64230    0
Name: Sentiment, Length: 37427, dtype: int64

In [19]:
df1['target']=f
df1[df1.target==0]

C:\Windows\TEMP/ipykernel_87044/2363693861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['target']=f


,Translated_Review,Sentiment,target
32,No recipe book Unable recipe book.,Negative,0
43,Waste time It needs internet time n ask calls ...,Negative,0
68,Faltu plz waste ur time,Negative,0
85,Crap Doesn't work,Negative,0
95,Boring. I thought actually just texts that's i...,Negative,0
...,...,...,...
64215,Horrible app. I wanted list property get aroun...,Negative,0
64216,Worst app. We get nothing Time waste . They up...,Negative,0
64220,"No response support team. After I login, unabl...",Negative,0
64226,"Dumb app, I wanted post property rent give opt...",Negative,0


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.Translated_Review,df1.target,test_size=0.1)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count

<33684x20231 sparse matrix of type '<class 'numpy.int64'>'
	with 524178 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB()

In [12]:
x_test_count = v.transform(X_test)
model.score(x_test_count,y_test)

0.7739780924392199

In [22]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train_count,y_train)
    return model.score(x_test_count, y_test)

In [23]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'Decision_Tree': {
        'model': DecisionTreeClassifier(),
        'params' : {
            
        }  
    },
    'KNN': {
        'model': KNeighborsClassifier(),
        'params' : {
            
        }  
    },
    'naive_bayes':{
        'model':MultinomialNB(),
        'params':{
            
        }
    }
}

In [25]:
scores = []
from sklearn.model_selection import GridSearchCV
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train_count,y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df_score = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_score

,model,best_score,best_params
0,svm,0.933054,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.879171,{'n_estimators': 10}
2,logistic_regression,0.934865,{'C': 5}
3,Decision_Tree,0.908117,{}
4,KNN,0.727318,{}
5,naive_bayes,0.788535,{}


In [26]:
lr_model= LogisticRegression(solver='liblinear',multi_class='auto', C=5)



In [27]:
lr_model.fit(X_train_count,y_train)

LogisticRegression(C=5, solver='liblinear')

In [32]:
review = [
    'Product is good',
    'Product is bad',
    'i like it',
    'ok',
    'so much expensive',
    'Worst app. We get nothing Time waste . They up',
    ' sde frfe dgtg dsvfv ewadwe ',
    'not ok',
    'broken',
    'Faltu plz waste ur time',
    'bad',
    '...'
]
reviews_count = v.transform(review)
result =lr_model.predict(reviews_count)
result1 = list(map( lambda x: 'Negative' if x==0 else ( 'Positive' if x==1 else 'Nutral' ) ,result  ))
result1

['Positive',
 'Negative',
 'Nutral',
 'Positive',
 'Negative',
 'Negative',
 'Nutral',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Nutral']

# Above you can see the results 

In [ ]:
#We make model pickel file

In [33]:
import pickle

with open('E:\Data Scientist\Amazon\Product_review.pickle','wb')as f:
    pickle.dump(lr_model,f)

# Summary ----------------------------------------------------------------------------------------------

In [ ]:
#Logistical regression having 93% accuracy